In [1]:
from hydradx.model.amm.omnipool_amm import OmnipoolState
from hydradx.model.amm.agents import Agent
from hydradx.model.processing import get_kraken_orderbook, get_omnipool_data, get_orderbooks_from_file, get_binance_orderbook
from hydradx.model.processing import get_omnipool_data_from_file
from hydradx.model.amm.centralized_market import CentralizedMarket
import hydradx.model.amm.arbitrage_agent as arb

from pprint import pprint

import json


Set `get_live_markets = True` to get live markets from Kraken. Set to `False` to use order books from file.

In [2]:
get_live_markets = True

In [3]:
# 2: Acala DAI
# 3: Acala WBTC
# 4: Acala WETH
# 5: DOT
# 9: ASTR
# 10: native USDT
# 11: iBTC
# 12: ZTG
# 13: CFG
# 14: BNC
# 15: vDOT
# 16: GLMR
# 17: INTR
# 18: Wormhole DAI
# 19: Wormhole WBTC
# 20: Wormhole WETH
# 100: 4-Pool

In [4]:
RPC="wss://rpc.hydradx.cloud"
            
if get_live_markets:
    asset_list, asset_map, tokens, fees = get_omnipool_data(RPC, archive=True)
else:
    asset_list, asset_map, tokens, fees = get_omnipool_data_from_file("data/input/")
    
# buffer2 = {((asset_map[k[0][0]], asset_map[k[0][1]]),k[1]): v for k, v in buffer.items()}

pprint(tokens)


{'4-Pool': {'LRNA': 47086.02652310476, 'liquidity': 1311979.3756877044},
 'ASTR': {'LRNA': 10052.163398343226, 'liquidity': 4211831.8680416895},
 'BNC': {'LRNA': 6745.580029038402, 'liquidity': 714260.0943161834},
 'CFG': {'LRNA': 9249.690375686125, 'liquidity': 601099.4256821818},
 'DAI': {'LRNA': 12440.493638168648, 'liquidity': 350638.26360525755},
 'DAI001': {'LRNA': 1407.281258338137, 'liquidity': 37585.53464434876},
 'DOT': {'LRNA': 69848.75125062505, 'liquidity': 396440.4654824683},
 'GLMR': {'LRNA': 5683.234178301682, 'liquidity': 626327.4502765426},
 'HDX': {'LRNA': 21840.650066987335, 'liquidity': 105139500.82263729},
 'INTR': {'LRNA': 9205.552574144767, 'liquidity': 12435850.696081579},
 'USDT': {'LRNA': 82723.60244090155, 'liquidity': 2305908.302395},
 'WBTC': {'LRNA': 41402.96203311108, 'liquidity': 31.39210041},
 'WBTC001': {'LRNA': 50.601798806111, 'liquidity': 0.04374934},
 'WETH': {'LRNA': 21886.072979851557, 'liquidity': 302.7630718334397},
 'WETH001': {'LRNA': 72708.

In [5]:
# arb_file = "arbconfig.txt"
arb_file = "arbconfig2.txt"
with open('config/' + arb_file, 'r') as json_file:
    cfg = json.load(json_file)
    
for d in cfg:
    d['tkns'] = tuple(d['tkns'])
    d['tkn_ids'] = tuple(d['tkn_ids'])
    d['order_book'] = tuple(d['order_book'])

ob_objs = {}
order_book_assets = {}
buffer = {}

if get_live_markets:
    for arb_cfg in cfg:
        tkn_pair = arb_cfg['order_book']
        # buffer[(arb_cfg['tkn_ids'],tkn_pair)] = arb_cfg['buffer']
        exchange = arb_cfg['exchange']
        if exchange not in ob_objs:
            ob_objs[exchange] = {}
        if tkn_pair not in ob_objs[exchange]:
            if exchange == 'kraken':
                ob_objs[exchange][tkn_pair] = get_kraken_orderbook(tkn_pair, archive=True)
            elif exchange == 'binance':
                ob_objs[exchange][tkn_pair] = get_binance_orderbook(tkn_pair, archive=True)
            else:
                raise ValueError("Exchange not supported")
else:
    ob_objs = get_orderbooks_from_file("data/input/")

for arb_cfg in cfg:
    tkn_pair = arb_cfg['order_book']
    exchange = arb_cfg['exchange']
    # buffer[(arb_cfg['tkn_ids'],tkn_pair)] = arb_cfg['buffer']
    for tkn in tkn_pair:
        if exchange not in order_book_assets:
            order_book_assets[exchange] = []
        if tkn not in order_book_assets[exchange]:
            order_book_assets[exchange].append(tkn) 
            
    arb_cfg['tkn_pair'] = (asset_map[arb_cfg['tkn_ids'][0]], asset_map[arb_cfg['tkn_ids'][1]])
            

                


In [6]:

cex_fees = {
    'kraken': 0.0016,
    'binance': 0.0010
}
# kraken_fee = 0.0016
# buffer = 0.0010

lrna_fee={asset: fees[asset]['protocol_fee'] for asset in asset_list}
asset_fee={asset: fees[asset]['asset_fee'] for asset in asset_list}

# lrna_fee = 0.0
# asset_fee = 0.0
# cex_fee = 0.0
# buffer = 0.0

cex_dict = {}
for exchange in ob_objs:
    cex_dict[exchange] = CentralizedMarket(
        order_book=ob_objs[exchange],
        asset_list=order_book_assets[exchange],
        trade_fee=cex_fees[exchange]
    )

op_state = OmnipoolState(
    tokens=tokens,
    lrna_fee = lrna_fee,
    asset_fee = asset_fee,
    preferred_stablecoin='USDT',
)
init_state = op_state.copy()


In [7]:
cex_dict1 = {exchange: cex_dict[exchange].copy() for exchange in cex_dict}
op_state1 = op_state.copy()
all_swaps1 = arb.get_arb_swaps_simple(op_state1, cex_dict1, cfg)

pprint(all_swaps1)
pprint(len(all_swaps1))

cex_dict2 = {exchange: cex_dict[exchange].copy() for exchange in cex_dict}
op_state2 = op_state.copy()
all_swaps2 = arb.get_arb_swaps(op_state2, cex_dict2, cfg)
print("strategy 2")
pprint(all_swaps2)
pprint(len(all_swaps2))

[{'cex': {'amount': 0.248,
          'buy_asset': 'DAI',
          'price': 2020.7091400000002,
          'sell_asset': 'ETH',
          'trade': 'sell'},
  'dex': {'amount': 0.248,
          'buy_asset': 'WETH001',
          'max_sell': 488.0219498242736,
          'price': 2021.72,
          'sell_asset': 'DAI001',
          'trade': 'buy'},
  'exchange': 'kraken'},
 {'cex': {'amount': 0.10516130400276186,
          'buy_asset': 'DAI',
          'price': 2019.655667,
          'sell_asset': 'ETH',
          'trade': 'sell'},
  'dex': {'amount': 0.10516130400276186,
          'buy_asset': 'WETH001',
          'max_sell': 210.85988992398975,
          'price': 2020.666,
          'sell_asset': 'DAI001',
          'trade': 'buy'},
  'exchange': 'kraken'},
 {'cex': {'amount': 0.08016552799987793,
          'buy_asset': 'XBT',
          'price': 36298.540199999996,
          'sell_asset': 'USDT',
          'trade': 'buy'},
  'dex': {'amount': 0.08016552799987793,
          'buy_asset': 'U

In [8]:
state1 = init_state.copy()
state2 = init_state.copy()
holdings = {asset: 1000000000 for asset in asset_list}
for exchange in cex_dict:
    for asset in cex_dict[exchange].asset_list:
        if asset not in holdings:
            holdings[asset] = 1000000000
init_agent = Agent(holdings=holdings, unique_id='bot')
agent1 = init_agent.copy()
agent2 = init_agent.copy()
cex_dict1 = {exchange: cex_dict[exchange].copy() for exchange in cex_dict}
cex_dict2 = {exchange: cex_dict[exchange].copy() for exchange in cex_dict}

arb.execute_arb(state1, cex_dict1, agent1, all_swaps1)
arb.execute_arb(state2, cex_dict2, agent2, all_swaps2)

asset_map = {'WETH': 'ETH',
             'XETH': 'ETH',
             'XXBT': 'BTC',
             'WBTC': 'BTC',
             'ZUSD': 'USD',
             'USDT': 'USD',
             'USDC': 'USD',
             'DAI': 'USD',
             'USDT001': 'USD',
             'DAI001': 'USD',
             'WETH001': 'ETH',
             'WBTC001': 'BTC',
             'iBTC': 'BTC',
             'XBT': 'BTC',}

profit1 = arb.calculate_profit(init_agent, agent1, asset_map)
profit2 = arb.calculate_profit(init_agent, agent2, asset_map)

asset_map_omnipool = {
    'ETH': 'WETH',
    'XETH': 'WETH',
    'XXBT': 'WBTC',
    'BTC': 'WBTC',
    'ZUSD': 'USDT',
    'USD': 'USDT',
    'USDC': 'USDT',
    'DAI': 'USDT',
    'USDT001': 'USDT',
    'DAI001': 'USDT',
    'WETH001': 'WETH',
    'WBTC001': 'WBTC',
    'iBTC': 'WBTC',
    'XBT': 'WBTC',
}


profit1_omnipool = arb.calculate_profit(init_agent, agent1, asset_map_omnipool)
profit2_omnipool = arb.calculate_profit(init_agent, agent2, asset_map_omnipool)

print("strategy 1 profit")
print(profit1)
print("strategy 1 profit at Omnipool spot")
print(sum([profit1_omnipool[asset] * state1.price(state1, asset, 'USDT') for asset in profit1_omnipool]))
print("strategy 2 profit")
print(profit2)
print("strategy 2 profit at Omnipool spot")
print(sum([profit2_omnipool[asset] * state2.price(state2, asset, 'USDT') for asset in profit2_omnipool]))
            

strategy 1 profit
{'4-Pool': 0, 'HDX': 0, 'USD': 42.36412274837494, 'ETH': 0.0, 'GLMR': 0.0, 'BTC': 0.00016105175018310547, 'BNC': 0.0, 'vDOT': 0, 'CFG': 0.0, 'DOT': 0.0, 'ZTG': 0, 'INTR': 0, 'ASTR': 0}
strategy 1 profit at Omnipool spot
48.22256953748127
strategy 2 profit
{'4-Pool': 0, 'HDX': 0, 'USD': 41.956172943115234, 'ETH': 0.0, 'GLMR': 0.0, 'BTC': 0.0004494190216064453, 'BNC': 0, 'vDOT': 0, 'CFG': 0.0, 'DOT': 0, 'ZTG': 0, 'INTR': 0, 'ASTR': 0}
strategy 2 profit at Omnipool spot
58.32878686811581


In [9]:
for swap in all_swaps2:
    if 'max_sell' in swap['dex']:
        profit = swap['cex']['price']*swap['cex']['amount'] - swap['dex']['max_sell']
        pprint(swap['dex']['buy_asset'] + ' in for ' + swap['dex']['sell_asset'] + ' out')
        num_asset = swap['dex']['sell_asset']
        pprint(str(swap['cex']['price']*swap['cex']['amount']) + ' ' + num_asset)
        pprint('profit: ' + str(profit) + ' ' + num_asset)
        print()
    elif 'min_buy' in swap['dex']:
        profit = swap['dex']['min_buy'] - swap['cex']['price']*swap['cex']['amount']
        pprint(swap['dex']['buy_asset'] + ' in for ' + swap['dex']['sell_asset'] + ' out')
        num_asset = swap['dex']['buy_asset']
        pprint(str(swap['cex']['price']*swap['cex']['amount']) + ' ' + num_asset)
        pprint('profit: ' + str(profit) + ' ' + num_asset)
        print()

'WETH001 in for DAI001 out'
'14.950584968000001 DAI001'
'profit: 0.57316154470975 DAI001'

'WETH001 in for DAI001 out'
'109.09778382 DAI001'
'profit: 3.8412162925270366 DAI001'

'WETH001 in for DAI001 out'
'604.0567059715468 DAI001'
'profit: 10.136078765817274 DAI001'

'USDT in for iBTC out'
'3718.373183865198 USDT'
'profit: 22.90819781040318 USDT'

'GLMR in for USDT out'
'26.866560000000003 USDT'
'profit: 0.28762050968170527 USDT'

'GLMR in for USDT out'
'670.7468541480191 USDT'
'profit: 4.071891622412977 USDT'

'WETH001 in for WBTC out'
'0.08582765132932235 WBTC'
'profit: 0.00044972903279831555 WBTC'

'USDT in for WBTC out'
'369.49561441419365 USDT'
'profit: 0.4159530692315343 USDT'

'CFG in for USDT out'
'3.337288521 USDT'
'profit: 0.007453062900158347 USDT'

'GLMR in for USDT out'
'23.767801511110044 USDT'
'profit: 0.041859724284389443 USDT'

'USDT in for iBTC out'
'102.54875722148537 USDT'
'profit: 0.07975617197634222 USDT'

'GLMR in for USDT out'
'6.655087586346724 USDT'
'profit: